# Nonlinear Amplifier
The following notebook demonstrates the function-baed nonlinear amplifier transform, which models the compressive effects of a nonlinear amplifier.

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.utils.defaults import default_dataset
import torchsig.transforms.functional as F
from torchsig.utils.dsp import multistage_polyphase_resampler

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

This function creates a QPSK signal which is used to test the nonlinear amplifier.

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 128.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["qpsk"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        noise_power_db=0.0,
        start_in_samples=0,
        seed=42,
    )
    signal = dataset.signal_generators[0]()

    # normalize, then scale data
    signal.data = F.normalize(data=signal.data, norm_order=2, flatten=False)
    signal.data = signal.data * scale

    return signal

The QPSK signal is created and then resampled.

In [ ]:
# test data
N = 4096
qpsk_bb_data = generate_qpsk_signal(num_iq_samples=N, scale=1.0).data

# upsample and rescale
qpsk_8x_data = multistage_polyphase_resampler(qpsk_bb_data, 8.0)
qpsk = qpsk_8x_data * np.sqrt(N)  # set qpsk mean_power to 1.0 W
qpsk_mean_power = np.mean(np.abs(qpsk) ** 2)
print(qpsk_mean_power)

The nonlinear amplifier is parameterized and then applied to the QPSK signal. The `pin_dBW` variable determines the range for which the amplifier is simulated over. Increasing the input power into an amplifier can reach a nonlinear region in which there are dimensing returns in terms of gain, or no increase in gain at all, which is the nonlinear effect.

The transform does not model the amplification effect itself (due to how SNR and power values are handled internally), but instead applies the nonlinear compressive effect. Therefore, a nominal gain of 1.0 is used with a saturation level `psat` of 1.0 W input power. The phase response is handled with the maximum phase difference according to the `phi_max` and the rate of change represented by `phi_slope`.

The result shows that as the input power `pin` increases the nonlinear compression effect is more pronounced. With `pin=-10dB` the effect is not noticable, when `pin=-5dB` some distortion can be seen around the edge of the constellation diagram, and when `pin=0dB` the four constellation points are moved off their originating points.

In [ ]:
# examine transform response across qpsk signal mean input powers
pin_dBW = np.arange(-30, 10 + 1, 1)  # [-30, +10] dBW signal mean input power range
pin = 10 ** (pin_dBW / 10)  # input power, linear units

gain = 1.0  # linear amplifier gain (ideal small-signal gain)
psat = 1.0  # fixed saturated output power level (W)
phi_max = -0.1  # relative phase shift maximum (radians)
phi_slope = 0.1  # linear slope of tanh phase response (W/rad)

pow_out = np.zeros((len(pin),))
ph_out = np.zeros((len(pin),))

for i, p in enumerate(pin):
    qpsk_in = qpsk * np.sqrt(
        pin[i] / 1.0
    )  # set qpsk mean power (original 1.0 linear power)
    psat_backoff = psat / np.mean(np.abs(qpsk_in) ** 2)  # set constant operating point
    response_data = F.nonlinear_amplifier(
        data=qpsk_in,
        gain=gain,
        psat_backoff=psat_backoff,
        phi_max=phi_max,
        phi_slope=phi_slope,
        auto_scale=False,
    )
    pow_out[i] = np.mean(np.abs(response_data) ** 2)  # mean power output estimate
    ph_out[i] = np.mean(
        np.angle(response_data) - np.angle(qpsk_in)
    )  # relative phase shift estimate

    if pin_dBW[i] == -10 or pin_dBW[i] == -5 or pin_dBW[i] == 0:
        # generate a plot
        fig = plt.figure(figsize=(12, 3))
        ax = fig.add_subplot(1, 2, 1)
        ax.plot(np.real(qpsk_in), np.imag(qpsk_in), label="in")
        ax.plot(np.real(response_data), np.imag(response_data), label="out")
        ax.set_aspect("equal")
        ax.set_title("Pin dB = " + str(pin_dBW[i]) + " dB")
        ax.legend(loc="upper right")
        ax.grid()
        ax = fig.add_subplot(1, 2, 2)
        ax.plot(np.real(qpsk_in), label="in")
        ax.plot(np.real(response_data), label="out")
        ax.legend(loc="upper right")
        ax.grid()

pin_log10 = 10 * np.log10(np.where(pin == 0, float("inf"), pin))
pout_log10 = 10 * np.log10(np.where(pow_out == 0, float("inf"), pow_out))

# plot pin/pout
fig = plt.figure(figsize=(8, 8))
plt.plot(pin_log10, pout_log10)
plt.title("Nonlinear Amplifier Model (Estimated AM/AM)", fontsize="large")
plt.xlabel("Pin (dB)", fontsize="large")
plt.ylabel("Pout (dB)", fontsize="large")
plt.show()

# plot pin/ph_out
fig = plt.figure(figsize=(8, 8))
plt.plot(pin_log10, ph_out)
plt.title("Nonlinear Amplifier Model (Estimated AM/PM)", fontsize="large")
plt.xlabel("Pin (dB)", fontsize="large")
plt.ylabel("$\\phi$ (rad)", fontsize="large")
plt.show()